In [1]:
import pandas as pd
import numpy as np
from numpy import math
from datetime import datetime, date, time
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [2]:
header_list = ['Date Time', 'Voltage', 'Current']
df = pd.read_csv('sensors_data.csv', names = header_list)
df['Power (W)'] = df['Voltage'] * df['Current']
df['Power (KW)'] = df['Power (W)'] / 1000
df['Date Time'] = pd.to_datetime(df['Date Time'])
df['Date'] = df['Date Time'].dt.date
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Date Time']).dt.time
df['Hour'] = pd.to_datetime(df['Date Time']).dt.hour
df['Time'] = df['Time'].astype(str)
# df['Hour'] = df['Hour'].astype(str)

In [3]:
df.head(5)

,Date Time,Voltage,Current,Power (W),Power (KW),Date,Time,Hour
0,2022-06-24 23:31:26,0.26855,0.0,0.0,0.0,2022-06-24,23:31:26,23
1,2022-06-25 00:31:26,0.26855,0.0,0.0,0.0,2022-06-25,00:31:26,0
2,2022-06-25 01:31:26,0.26855,0.0,0.0,0.0,2022-06-25,01:31:26,1
3,2022-06-25 01:32:26,0.26855,0.0,0.0,0.0,2022-06-25,01:32:26,1
4,2022-06-25 01:33:26,0.31738,0.0,0.0,0.0,2022-06-25,01:33:26,1


In [4]:
rearrange_columns = ['Date Time', 'Date', 'Time', 'Hour', 'Voltage', 'Current', 'Power (W)', 'Power (KW)']
df = df[rearrange_columns]
df.tail(5)

,Date Time,Date,Time,Hour,Voltage,Current,Power (W),Power (KW)
2349,2022-06-26 18:28:20,2022-06-26,18:28:20,18,13.79395,0.16782,2.314901,0.002315
2350,2022-06-26 18:29:20,2022-06-26,18:29:20,18,13.84277,0.17817,2.466366,0.002466
2351,2022-06-26 18:30:20,2022-06-26,18:30:20,18,13.84277,0.18469,2.556621,0.002557
2352,2022-06-26 18:31:21,2022-06-26,18:31:21,18,13.86719,0.20963,2.906979,0.002907
2353,2022-06-26 18:32:21,2022-06-26,18:32:21,18,13.91602,0.23111,3.216131,0.003216


In [5]:
unique_date = df['Date'].unique()
unique_date

array(['2022-06-24T00:00:00.000000000', '2022-06-25T00:00:00.000000000',
       '2022-06-26T00:00:00.000000000'], dtype='datetime64[ns]')

In [6]:
filter_today_values = df[df['Date'] == unique_date[-1]][['Date', 'Hour', 'Power (KW)']]
today_hourly_values = filter_today_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
length_today_hourly_values = len(today_hourly_values)
length_today_hourly_values
# today_hourly_values

19

In [7]:
filter_daily_values = df[(df['Date'] > '2022-06-24') & (df['Date'] <= unique_date[-2])][['Date', 'Hour', 'Power (KW)']]
filter_daily_values

,Date,Hour,Power (KW)
1,2022-06-25,0,0.0
2,2022-06-25,1,0.0
3,2022-06-25,1,0.0
4,2022-06-25,1,0.0
5,2022-06-25,1,0.0
...,...,...,...
1239,2022-06-25,23,0.0
1240,2022-06-25,23,0.0
1241,2022-06-25,23,0.0
1242,2022-06-25,23,0.0


In [8]:
daily_hourly_values = filter_daily_values.groupby(['Date', 'Hour'])['Power (KW)'].sum().reset_index()
daily_hourly_values

,Date,Hour,Power (KW)
0,2022-06-25,0,0.000000
1,2022-06-25,1,0.000000
2,2022-06-25,2,0.000000
3,2022-06-25,3,0.000000
4,2022-06-25,4,0.000000
5,2022-06-25,5,0.006974
6,2022-06-25,6,0.087022
7,2022-06-25,7,0.194788
8,2022-06-25,8,0.896399
9,2022-06-25,9,1.671361


In [9]:
# daily_values = daily_values[daily_values['Date'] > '2022-06-11']

In [10]:
header_list = ['Date', 'Time', 'SolarIrradiance (W/m2)', 'weather status', 'Temp (°C)', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)',
              'Direction', 'Hum (%)', 'Visibility (km)', 'UVIndex', 'UVIndexText', 'PreProbability (%)', 'RainProbability (%)',
              'CloudCover (%)']
weather_data = pd.read_csv('hourly_weather_forecasted_data.csv', names = header_list, encoding= 'unicode_escape')
weather_data.drop(['Date', 'Time', 'RealFeelTemp (°C)', 'DewPoint (°C)', 'Wind (km/h)', 'Direction', 'Visibility (km)', 'UVIndex', 
                   'UVIndexText', 'PreProbability (%)', 'RainProbability (%)', 'weather status'], axis = 1, inplace = True)
weather_data.tail(14)

,SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
34,131.8,16.1,52,78
35,174.8,16.9,48,63
36,167.4,18.8,43,70
37,170.0,19.6,41,70
38,167.8,19.9,40,70
39,150.0,19.8,38,75
40,121.0,19.2,41,84
41,132.1,19.4,39,70
42,110.5,18.9,38,70
43,84.0,18.2,40,70


In [11]:
df1 = pd.concat([daily_hourly_values, weather_data], axis = 1)
df1.drop(['Date', 'Hour'], axis = 1, inplace = True)
df1.loc[df1['SolarIrradiance (W/m2)'] == 0, ['Temp (°C)', 'Hum (%)', 'CloudCover (%)']] = 0
# df1.to_csv('solar_energy_predictions.csv')
df1.tail(14)

,Power (KW),SolarIrradiance (W/m2),Temp (°C),Hum (%),CloudCover (%)
34,NaN,131.8,16.1,52,78
35,NaN,174.8,16.9,48,63
36,NaN,167.4,18.8,43,70
37,NaN,170.0,19.6,41,70
38,NaN,167.8,19.9,40,70
39,NaN,150.0,19.8,38,75
40,NaN,121.0,19.2,41,84
41,NaN,132.1,19.4,39,70
42,NaN,110.5,18.9,38,70
43,NaN,84.0,18.2,40,70


In [12]:
count_total_rows = len(df1) - 24

In [13]:
independent_columns = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']][0:count_total_rows]


In [14]:
dependent_column = df1['Power (KW)'][0:count_total_rows]

In [15]:
# st_independent_columns = StandardScaler()
# st_dependent_column = StandardScaler()

In [16]:
# x_independent_columns = [st_independent_columns.fit_transform(independent_columns)]
# y_dependent_column = [st_dependent_column.fit_transform(dependent_column)]

In [17]:
dec_tree_regressor = DecisionTreeRegressor(random_state = 10)
dec_tree_regressor.fit(independent_columns, dependent_column)

DecisionTreeRegressor(random_state=10)

#### Is model well fit or not?

In [18]:
r_sq = dec_tree_regressor.score(independent_columns, dependent_column)
r_sq

1.0

In [19]:
forcasted_data = df1[['SolarIrradiance (W/m2)', 'Temp (°C)', 'Hum (%)', 'CloudCover (%)']].tail(24)

In [20]:
return_array = list(dec_tree_regressor.predict(forcasted_data))

In [21]:
now = datetime.now()
date = now.strftime('%Y-%m-%d')
current_date_24 = [date, date, date, date, date, date, date, date, date, date, date, date, date, date, date, date,
                date, date, date, date, date, date, date, date]
current_date_12 = [date, date, date, date, date, date, date, date, date, date, date, date]

In [22]:
hours_24 = list(daily_hourly_values['Hour'][0:24])
hours_12 = list(daily_hourly_values['Hour'][0:12])

In [23]:
data_dict = {'Date': current_date_24, 'Hour': hours_24, 'Power (KW)': return_array}

In [24]:
data_dataframe = pd.DataFrame(data_dict)
data_dataframe

,Date,Hour,Power (KW)
0,2022-06-26,0,0.000000
1,2022-06-26,1,0.000000
2,2022-06-26,2,0.000000
3,2022-06-26,3,0.000000
4,2022-06-26,4,0.000000
5,2022-06-26,5,0.006974
6,2022-06-26,6,0.036215
7,2022-06-26,7,0.153809
8,2022-06-26,8,1.219501
9,2022-06-26,9,2.112699


## Evaluating the model

#### MSE = mean_squared_error(actual value, predicted value)

In [25]:
mean_sq_error = metrics.mean_squared_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_sq_error

0.598760145906159

In [26]:
root_mean_sq_error = np.sqrt(mean_sq_error)
root_mean_sq_error

0.7737959329863132

In [27]:
mean_ab_error = metrics.mean_absolute_error(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
mean_ab_error

0.4523486954131737

#### R squared

In [28]:
r_squared = metrics.r2_score(today_hourly_values['Power (KW)'], data_dataframe['Power (KW)'].head(length_today_hourly_values))
r_squared

0.13186081277156203